In [1]:
import torch
from torch import nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
print(X.device)
net(X)

cpu


tensor([[0.2259],
        [0.2082]], grad_fn=<AddmmBackward0>)

In [4]:
print(net[0].state_dict())

OrderedDict([('weight', tensor([[ 0.2102, -0.4577, -0.3507,  0.4310],
        [-0.0788,  0.4388,  0.2121, -0.3962],
        [ 0.1750,  0.4589,  0.3715,  0.0409],
        [ 0.4251, -0.2397, -0.1558,  0.3706],
        [ 0.0058, -0.4892,  0.2702, -0.4865],
        [ 0.4607,  0.0727, -0.0765,  0.4751],
        [ 0.3020, -0.1550, -0.0874,  0.1621],
        [ 0.2859, -0.3179, -0.3879,  0.2661]])), ('bias', tensor([-0.1455,  0.4873, -0.1597, -0.4965,  0.0251,  0.1695,  0.4943, -0.3737]))])


In [9]:
print(type(net[0].bias))
print(net[0].bias[1])

<class 'torch.nn.parameter.Parameter'>
tensor(0.4873, grad_fn=<SelectBackward0>)


In [11]:
net[2].weight.grad==None

True

In [12]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [13]:
print(*[(name,param.shape) for name,param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [15]:
net.state_dict()['0.bias'].data

tensor([-0.1455,  0.4873, -0.1597, -0.4965,  0.0251,  0.1695,  0.4943, -0.3737])

In [16]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
    net=nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
    return net

rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.0654],
        [0.0654]], grad_fn=<AddmmBackward0>)

In [17]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [18]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([-0.0010, -0.0034,  0.0058, -0.0004]), tensor(0.))

In [19]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [20]:
def xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.3751,  0.3451,  0.6854, -0.5934])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [22]:
def my_init(m):
    if type(m)==nn.Linear:
        print("Init",*[(name,param.shape) for name,param in m.named_parameters()])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data*=m.weight.data.abs()>=5

net.apply(my_init)
net[0].weight[:2]

Init ('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
Init ('weight', torch.Size([1, 8])) ('bias', torch.Size([1]))


tensor([[ 5.8742, -5.5201, -7.8142,  0.0000],
        [ 8.2466, -0.0000, -6.7004,  0.0000]], grad_fn=<SliceBackward0>)

In [23]:
net[0].weight.data[:] += 1
net[0].weight.data[0,0]=42
net[0].weight.data[0]

tensor([42.0000, -4.5201, -6.8142,  1.0000])

In [ ]:
shared=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))

net(X)
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0,0]=100
print(net[2].weight.data[0]==net[4].weight.data[0])